In [ ]:
!pip3 install uproot
!pip3 install mplhep

In [ ]:
import uproot
import awkward
import mplhep as hep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd

In [ ]:
def DeltaPhi(phi):
  """
  phi: pandas series
  """
  for i in range(len(phi)):
    if phi[i] >= np.pi:
      phi[i] -= 2*np.pi
    elif phi[i] < -1*np.pi:
      phi[i] += 2*np.pi 

  return phi  

def permutations():

  p = []
  for i in range(1,7):
    for j in range(1,7):
      if i != j and (j,i) not in p:
        p.append((i,j))

  return p

def invariant_mass(pt1, pt2, eta1, eta2, phi1, phi2):

  return np.sqrt(2*pt1*pt2*(np.cosh(eta1 - eta2) - np.cos(phi1 - phi2)))

def add_invariant_mass(df):

  # invariant mass
  df["inv_mass"] = invariant_mass(df["jet_pt1"],
                                  df["jet_pt2"],
                                  df["jet_eta1"],
                                  df["jet_eta2"],
                                  df["jet_phi1"],
                                  df["jet_phi2"])
  
  # max invariant mass
  aux_df = pd.DataFrame(index=np.arange(len(df)))

  for i,j in permutations():
    aux_df[f"{i,j}"] =  invariant_mass(df[f"jet_pt{i}"],
                                        df[f"jet_pt{j}"],
                                        df[f"jet_eta{i}"],
                                        df[f"jet_eta{j}"],
                                        df[f"jet_phi{i}"],
                                        df[f"jet_phi{j}"])
    
  df["max_inv_mass"] = np.max(aux_df, axis=1)

  return df

def delta_phi_met_jet(df):

  aux_df = pd.DataFrame(index=np.arange(len(df)))

  for i in range(1,7):
    phi = df[f"jet_phi{i}"] - df["missinget_phi"]
    aux_df[f"{i}"] = np.abs(DeltaPhi(phi))  
    
  df["min_delta_phi_met_jet"] = aux_df.min(axis=1)

  return df

def CutFlow(sig_cut, bac_cut, n_cuts=8, ns=50000, nb=50000, L=25000, sigmab=72.38, sigmas=13.76):
    """
    Returns the cut flow tables

    sig_cut,bac_cut:  dictionaries with the signal 
                      and background cuts
    cut_keys:         dictionary with the cuts keys
    n_cuts:           number of available cuts 
    L:                integrated luminosity in pb^-1
    
    ns,nb:            number of signal and background events
    
    sigmas,sigmab:    cross sections for the signal and
                      background events in pb

    """
    deltaEta_cut_value = 0
    invmass_cut_value = 1000
    met_cut_value = 200
    h_cut_value = 200
    delta_phi_cut_value = 0.5

    cut0 = "No cuts"
    cut1 = f"$\mathbf{{H_t >}}{h_cut_value}$"
    cut2 = "$n_jets \geq 2$"
    cut3 = f"$\mathbf{{P_T>30}}$, $|\eta(j)|<5$"
    cut4 = "$\eta (j_1) * \eta (j_2) < 0$"
    cut5 = f"$|\Delta \eta (j_1,j_2)| > {deltaEta_cut_value}$"
    cut6 = f"$\mathbf{{\text{{max}}M(j_1,j_2) >}}{invmass_cut_value}$"
    cut7 = f"|$\Delta\phi(\text{{MET}},j)| > {delta_phi_cut_value}$"

    math_cuts = [cut0,
                 cut1,
                 cut2,
                 cut3,
                 cut4,
                 cut5,
                 cut6,
                 cut7]

    cut_keys = {f"cut{i}":key for i,key in zip(range(8), math_cuts)}
    
    # number of applied cuts
    N_cuts = np.count_nonzero(list(s_cuts.values()))

    # modified cut dics
    s_cut = {key:value for key, value in zip(sig_cut.keys(), sig_cut.values()) if value !=0}
    b_cut = {key:value for key, value in zip(bac_cut.keys(), bac_cut.values()) if value !=0}
    c_keys = {f"cut{i}":cut_keys[f"cut{i}"] for i in range(N_cuts)}

    # weights for signal and background events
    ws = L*sigmas/ns
    wb = L*sigmab/nb

    # dataframes
    s1 = pd.Series(s_cut, dtype="float64") if N_cuts!=8 else pd.Series(sig_cut, dtype="float64")
    s2 = pd.Series(b_cut, dtype="float64") if N_cuts!=8 else pd.Series(bac_cut, dtype="float64")
    s3 = pd.Series(c_keys) if N_cuts!=8 else pd.Series(cut_keys) 

    # cut flow tables
    df1 = pd.concat([s3,s1,s2], axis=1, keys=["${\textbf{[bold]: GeV}}$","S","B"])
    
    df1["Z"] = ws*df1["S"]/np.sqrt(ws*df1["S"]+wb*df1["B"]) 

    df2 = pd.DataFrame(index=[f"cut{i}" for i in range(N_cuts)],
                       columns=["${\\textbf{[bold]: GeV}}$","s_c","s_r","b_a","b_r"])

    for i in range(N_cuts):
      df2.iloc[i,0] = cut_keys[f"cut{i}"]

    for i in range(1,5):
      df2.iloc[0,i] = 1

    for i in range(1, N_cuts):
      df2.iloc[i,1] = df1.iloc[i,1]/df1.iloc[0,1]
      df2.iloc[i,3] = df1.iloc[i,2]/df1.iloc[0,2]
      df2.iloc[i,2] = df1.iloc[i,1]/df1.iloc[i-1,1]
      df2.iloc[i,4] = df1.iloc[i,2]/df1.iloc[i-1,2]

    return df1, df2

In [ ]:
def df_lower_names(df):

  df.columns = [name.lower().replace(".","_") for name in df.columns]

  return df

def pad(jet):

  for i in range(len(jet)):
    while len(jet[i]) < 6:
      jet[i] = np.hstack((jet[i], [None]))

  return jet

def read_jet_branch(data, branch_name):

  return pad([jet[:6] for jet in data.array(branch_name)])

def get_jet_dic(jet, name):

  jets = {f"{name}{i}":[] for i in range(1,7)} 

  for jet in jet:
    for i in range(6):
      jets[f"{name}{i+1}"].append(jet[i])

  return jets

  {"jet_pt1":[], "jet_pt2":[]}

def get_jet_series(jet_var, name):

  keys = [name+f"{i}" for i in range(1,7)]
  
  return [pd.Series(jet_var[name], name=name) for name in keys]

def get_jet_dataframe(data, *branch_names):

  jet_keys = [key for key in branch_names if "Jet" in key]
  jets = {key:read_jet_branch(data, key) for key in jet_keys}

  names = [name for name in jets]

  jet_columns = {name:get_jet_series(get_jet_dic(jets[name], name), name) for name in names}

  df = pd.DataFrame(index=np.arange(len(jets[names[0]])))

  for key in jet_columns.keys():
    for i in range(6):
      df = df.join(jet_columns[key][i])

  # adding number of jets to dataframe
  n_jets = [np.count_nonzero(jet) for jet in jets[jet_keys[0]]]
  df["n_jets"] = n_jets

  return df_lower_names(df) 

def get_nonjet_dataframe(data, *branch_names):

  df_keys = [key for key in branch_names if "Jet" not in key]
  df = data.arrays(branches=[*df_keys], outputtype=pd.DataFrame).astype("float64")

  return df_lower_names(df)

def get_dataframe(data, *branch_names):

  # non-jet df
  nonjet_df = get_nonjet_dataframe(data, *branch_names)

  # jet df
  jet_df = get_jet_dataframe(data, *branch_names)

  # joining dataframes
  df = nonjet_df.join(jet_df)

  # adding H
  jets_pt = [f"jet_pt{i}" for i in range(1,7)]
  df["H"] = df[jets_pt].sum(axis=1)

  if "Jet.Phi" in branch_names:
    pt1, pt2 = df["jet_pt1"], df["jet_pt2"] 
    eta1, eta2 = df["jet_eta1"], df["jet_eta2"]
    phi1, phi2 = df["jet_phi1"], df["jet_phi2"]

    # adding invariant mass 
    df = add_invariant_mass(df)  

    # adding deltaPhi for leading jets
    df["delta_phi"] = np.abs(DeltaPhi(phi1 - phi2))

    # adding min deltaPhi for met and jets
    df = delta_phi_met_jet(df)

  return df

In [ ]:
# Cuts

def cut1(df, cuts):

  # H > 200
  mask = df.H > 200
  df = df.loc[mask,:]
  cuts["cut1"] = len(df)

  return df

def cut2(df, cuts):

  # at least two jets per event 
  mask = df.n_jets >=2
  df = df.loc[mask,:]
  cuts["cut2"] = len(df)

  return df

def cut3(df, cuts):

  # pt > 30 GeV and |eta| < 5 for leading and subleading jets  
  mask = (df.jet_pt1 > 30) & (df.jet_pt2 > 30) & (abs(df.jet_eta1) < 5) & (abs(df.jet_eta2) < 5) 
  df = df.loc[mask,:]
  cuts["cut3"] = len(df)

  return df

def cut4(df, cuts):

  # leading jets in opposite hemispheres
  mask = df.jet_eta1 * df.jet_eta2 < 0
  df = df.loc[mask,:]
  cuts["cut4"] = len(df)

  return df

def cut5(df, cuts):

  # |DeltaPhi| >= 2.3 for leading jets
  mask = abs(df.delta_phi) >= 2.3
  df = df.loc[mask,:]
  cuts["cut5"] = len(df)

  return df

def cut6(df, cuts):

  # max invariant mass >= 1000
  mask = df.max_inv_mass >= 1000
  df = df.loc[mask,:]
  cuts["cut6"] = len(df)

  return df

def cut7(df, cuts):

  mask = df.min_delta_phi_met_jet > 0.5
  df = df.loc[mask,:]
  cuts["cut7"] = len(df)

  return df

def Cuts(df, n_cuts=3):

  cuts = {"cut0":len(df)}

  while True:

    df = cut1(df, cuts) 
    if n_cuts == 1: break
    
    df = cut2(df, cuts) 
    if n_cuts == 2: break

    df = cut3(df, cuts) 
    if n_cuts == 3: break
  
    df = cut4(df, cuts)
    if n_cuts == 4: break

    df = cut5(df, cuts)
    if n_cuts == 5: break  

    df = cut6(df, cuts)
    if n_cuts == 6: break

    df = cut7(df, cuts)
    if n_cuts == 7: break

  return df, cuts

## Reading the data

In [ ]:
signal_path = "/content/drive/Shared drives/VBF_DM_UdeA/Samples/DMSimp_spin0/DMSimpSpin0_MY5000_MX1000_07042020.root"
back_path = "/content/drive/Shared drives/VBF_DM_UdeA/Samples/ZjetstoNuNu/santiago_run_02.root"

signal = uproot.open(signal_path)["Delphes"]
background = uproot.open(back_path)["Delphes"]

In [ ]:
ns = signal.numentries
nb = background.numentries
ns, nb  

In [ ]:
signal_df = get_dataframe(signal, "Jet.PT", "MissingET.Phi", "MissingET.MET", "Jet.Eta", "MissingET.Eta", "Jet.Phi")
signal_df.head()

In [ ]:
background_df = get_dataframe(background, "Jet.PT", "MissingET.Phi", "MissingET.MET", "Jet.Eta", "MissingET.Eta", "Jet.Phi")
background_df.head()

## Applying cuts

In [ ]:
s_df, s_cuts = Cuts(signal_df, n_cuts=7) 

In [ ]:
b_df, b_cuts = Cuts(background_df, n_cuts=7)

In [ ]:
t1, t2 = CutFlow(s_cuts, b_cuts)

In [ ]:
t1

In [ ]:
t2

In [ ]:
# set a CMS style 
plt.style.use(hep.style.CMS)

fig, ax = plt.subplots(1, figsize=(9,6))

#ax.hist(signal_df.met_met, histtype="step" ,bins=100, label="s", density=True)
ax.hist(b_df.max_inv_mass, histtype="step", bins=100, label="b")#, density=True);
ax.set_xlim(0,3000)

# set the figure title with CMS style info
hep.cms.label(data=False, paper=False, year='2020', ax=ax)
# set the figure title with (CMS style) some text 
#hep.cms.text("Preliminary")
plt.legend();